### **01_hello_world.ipynb**
### **Hello World Pipeline**

* ##### 01 - Install packages
* ##### 02 - Import packages
* ##### 03 - Create tasks
* ##### 04 - Create pipeline
* ##### 05 - Create pipeline yaml
* ##### 06 - Create pipeline run

### 01 - Install packages

In [ ]:
!pip install kfp

### 02 - Import packages

In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

import kfp

from components.create_hello_world_message   import create_hello_world_message
from components.create_odds_or_evens_message import create_odds_or_evens_message
from components.create_random_number         import create_random_number

### 03 - Create tasks

In [ ]:
task_base_image = 'registry.access.redhat.com/ubi9/python-311'

In [ ]:
create_hello_world_message_op = kfp.dsl.component(
    func       = create_hello_world_message,
    base_image = task_base_image
)

In [ ]:
create_random_number_op = kfp.dsl.component(
    func       = create_random_number,
    base_image = task_base_image
)

In [ ]:
create_odds_or_evens_message_op = kfp.dsl.component(
    func       = create_odds_or_evens_message,
    base_image = task_base_image
)

### 04 - Create pipeline

In [ ]:
pipeline_name        = '01_hello_world'
pipeline_description = 'Hello World Pipeline'

In [ ]:
@kfp.dsl.pipeline(
    name        = pipeline_name,
    description = pipeline_description
)
def pipeline(
    name    : str,
    minimum : int,
    maximum : int
):

    create_hello_world_message_task = create_hello_world_message_op(
        name = name
    )

    create_random_number_task = create_random_number_op(
        minimum = minimum,
        maximum = maximum
    )

    create_odds_or_evens_message_op(
        hello_world_message = create_hello_world_message_task.output,
        random_number       = create_random_number_task.output
    )

### 05 - Create pipeline yaml

In [ ]:
pipeline_package_path = os.path.join('yaml', f'{ pipeline_name }.yaml')

In [ ]:
kfp.compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path  = pipeline_package_path
)

### 06 - Create pipeline run

In [ ]:
kubeflow_host  = '<kubeflow_host>'
kubeflow_token = '<kubeflow_token>'

In [ ]:
pipeline_arguments = {
    'name'    : 'ML Pipelines',
    'minimum' : 1,
    'maximum' : 100
}

In [ ]:
kfp.client.Client(host = kubeflow_host, existing_token = kubeflow_token).create_run_from_pipeline_package(
    pipeline_file = pipeline_package_path,
    arguments     = pipeline_arguments
)